##**Mounting the Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##**Unzipping the dataset**

In [ ]:
!unzip "/content/drive/MyDrive/FaceMask_Detection.zip" -d "/content/"

##**Importing the Libraries**

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,  BatchNormalization
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam

##**Setting up the directories**

In [ ]:
# Dimension of our images.
img_width, img_height = 150, 150

#Setting up the directories
train_data_dir = '/content/Face Mask Dataset/Train'
validation_data_dir = '/content/Face Mask Dataset/Validation'

In [ ]:
input_shape = (img_width, img_height, 3)
epochs = 50
batch_size = 32

##**Data Augmentation**

In [ ]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1./ 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

#this generates batches of augment data for training
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 10000 images belonging to 2 classes.


In [ ]:
# this is the augmentation configuration we will use for validating
val_datagen = ImageDataGenerator(rescale=1./255)

#this generates batches of augment data for validating
validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 800 images belonging to 2 classes.


##**Building the Model**

###**Loading the Base Model**

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

mobilenet = MobileNetV2(weights = "imagenet",include_top = False,input_shape=(150,150,3))

9420800/9406464 [==============================] - 0s 0us/step


In [ ]:
mobilenet.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 75, 75, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 75, 75, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 75, 75, 32)   0           ['bn_Conv1[0][

###**Adding the classifier part**

In [ ]:
for layer in mobilenet.layers:
    layer.trainable = False

In [ ]:
model = Sequential()
model.add(mobilenet)
model.add(Flatten())
model.add(Dense(1,activation="sigmoid"))

##**Compiling the Model**

In [ ]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics ="accuracy")

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 5, 5, 1280)       2257984   
 ional)                                                          
                                                                 
 flatten (Flatten)           (None, 32000)             0         
                                                                 
 dense (Dense)               (None, 1)                 32001     
                                                                 
Total params: 2,289,985
Trainable params: 32,001
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:
#Setting up the batchsizes.
nb_train_samples = 10000
nb_validation_samples = 800

In [ ]:
print(train_generator.class_indices)

{'WithMask': 0, 'WithoutMask': 1}


##**EarlyStopping Callback**

In [ ]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1,patience=2,restore_best_weights=True)

##**Training the model**

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,callbacks=[es])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/50
312/312 [==============================] - 106s 238ms/step - loss: 0.1070 - accuracy: 0.9788 - val_loss: 0.1461 - val_accuracy: 0.9800
Epoch 2/50
312/312 [==============================] - 73s 233ms/step - loss: 0.0412 - accuracy: 0.9933 - val_loss: 0.2931 - val_accuracy: 0.9688
Epoch 3/50
312/312 [==============================] - 73s 235ms/step - loss: 0.0278 - accuracy: 0.9952 - val_loss: 0.0359 - val_accuracy: 0.9925
Epoch 4/50
312/312 [==============================] - 73s 234ms/step - loss: 0.0461 - accuracy: 0.9949 - val_loss: 0.1187 - val_accuracy: 0.9875
Epoch 5/50
312/312 [==============================] - 73s 233ms/step - loss: 0.0171 - accuracy: 0.9979 - val_loss: 0.0767 - val_accuracy: 0.9912
Epoch 00005: early stopping


##**Saving the Model Architecture and the Weights**

In [ ]:
model.save_weights('FaceMask_Detection_second_try_three.h5')

In [ ]:
with open('model_architecture_FaceMask_Detection_second_try_three.json','w') as f:
    f.write(model.to_json())

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
